In [1]:
import config

In [2]:
from transformers import AutoTokenizer

model_path = str(config.GPT2_PATH)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/opt/homebrew/anaconda3/envs/rlhf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

dataset_path = str(config.SST2_PATH)
dataset = load_dataset(dataset_path)
print(dataset)

ds_train, ds_val = dataset['train'], dataset['validation']
print(ds_train[4])

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})
{'idx': 4, 'sentence': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'label': 0}


In [4]:
REWARD_TOKEN_ID = tokenizer.eos_token_id
print(REWARD_TOKEN_ID)


def tokenize(batch):
    # 提取出文本内容
    outputs = tokenizer(batch['sentence'])
    # 每条数据一个评分，初始化为 0 。
    outputs['score'] = [0] * len(outputs['input_ids'])
    # 对每条数据的最后的reward token进行评分
    outputs['score_index'] = [0] * len(outputs['input_ids'])
    for i in range(len(outputs['input_ids'])):
        # 第 i 条数据的末尾添加一个 eos token，作为reward token
        outputs['input_ids'][i].append(REWARD_TOKEN_ID)
        # reward token的掩码设置为 1 。
        outputs['attention_mask'][i].append(1)
        # 正向情感的文本评分为 1 。负向情感的评分为 0 。
        outputs['score'][i] = float(batch['label'][i])
        # 对 reward token 进行评分，也就是评分的索引为 reward token 的索引。
        outputs['score_index'][i] = len(outputs['input_ids'][i]) - 1
    return outputs


map_kwargs = {
    "batched": True,
    "batch_size": 512,
    "remove_columns": ['idx', 'sentence', 'label']
}

tokenized_dataset_train = ds_train.map(tokenize, **map_kwargs)
tokenized_dataset_val = ds_val.map(tokenize, **map_kwargs)

print(tokenized_dataset_train[4])

50256


Map: 100%|██████████| 872/872 [00:00<00:00, 61496.34 examples/s]

{'input_ids': [261, 262, 5290, 15827, 12, 1659, 12, 1169, 12, 1008, 9310, 35478, 20954, 262, 28303, 714, 47478, 469, 510, 220, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'score': 0.0, 'score_index': 20}


In [5]:
tokenized_dataset_train.set_format(type='torch')
tokenized_dataset_val.set_format(type='torch')

print(tokenized_dataset_train[4])

{'input_ids': tensor([  261,   262,  5290, 15827,    12,  1659,    12,  1169,    12,  1008,
         9310, 35478, 20954,   262, 28303,   714, 47478,   469,   510,   220,
        50256]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'score': tensor(0.), 'score_index': tensor(20)}


In [6]:
tokenized_dataset_train = tokenized_dataset_train.filter(lambda x: len(x['input_ids']) > 6)
tokenized_dataset_val = tokenized_dataset_val.filter(lambda x: len(x['input_ids']) > 6)

print(len(tokenized_dataset_train))

Filter: 100%|██████████| 872/872 [00:00<00:00, 88046.05 examples/s]

49401


In [7]:
from model import GPT2RewardHead

model = GPT2RewardHead(model_path)

In [8]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# 还是将 eos token 作为 pad token
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorWithPadding(tokenizer)
dataloader_params = {
    'batch_size': 32,
    'shuffle': True,
    'collate_fn': data_collator
}
train_dataloader = DataLoader(tokenized_dataset_train, **dataloader_params)
val_dataloader = DataLoader(tokenized_dataset_val, **dataloader_params)

batch = next(iter(train_dataloader))
print(batch.keys())

print(batch['input_ids'][1])
print(batch['attention_mask'][1])
print(batch['score'][1])
print(batch['score_index'][1])
print(tokenizer.decode(batch['input_ids'][1]))
print(batch['attention_mask'][1].nonzero()[-1])

KeysView({'input_ids': tensor([[ 1169,  2818,  1986,  ..., 50256, 50256, 50256],
        [10594,   307,   308,  ..., 50256, 50256, 50256],
        [11246,  5141,   508,  ..., 50256, 50256, 50256],
        ...,
        [13959,  1014,   832,  ..., 50256, 50256, 50256],
        [   11,   788,   484,  ..., 50256, 50256, 50256],
        [ 8988,   572,   355,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'score': tensor([1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.]), 'score_index': tensor([14, 20, 16,  9, 11,  6,  6, 30, 56, 47, 12, 28, 14,  6,  8,  6,  8,  6,
        33,  9, 12,  6, 12, 11, 22,  9,  7,  6, 21, 36, 22, 18])})
tensor([10594,   307,   308, 10316,   284,  2687,   407,  27

In [9]:
outputs = model(batch['input_ids'], batch['attention_mask'])
print(outputs.shape)

torch.Size([32, 57])


In [10]:
import torch
import torch.nn as nn

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# 二分类交叉熵损失
criterion = nn.BCELoss()
num_epochs = 1  # N+ Implementation Detail paper

In [11]:
def validate():
    model.eval()
    total_loss = 0
    for i, batch in enumerate(val_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        with torch.no_grad():
            # 对输出进行评分
            scores = model(**model_inputs)
            # 批次中每条数据的索引
            batch_indices = torch.arange(scores.shape[0])
            # 根据索引拿出评分，也就是reward token的评分
            score = scores[batch_indices, inputs['score_index']]
            # 目标评分，0 或者 1 。
            target = inputs['score']
            # 计算误差
            loss = criterion(score, target)
        total_loss += loss.item()
    print('validation loss:', total_loss / len(val_dataloader))

In [12]:
model.to(device)

validate()
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        inputs = batch.to(device)
        model_inputs = {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
        loss = criterion(score, target)
        # 三部曲：清空梯度 ⟶ 反向传播计算梯度 ⟶ 更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print(loss.item())
    validate()

validation loss: 4.739590099879673
3.476541519165039
0.35558801889419556
validation loss: 0.24328557381938612


In [13]:
torch.save(model.state_dict(), str(config.REWARD_MODEL_PATH))

In [14]:
validate()

validation loss: 0.24330428748258523


In [15]:
from sklearn.metrics import confusion_matrix

model.eval()

all_predictions = []
all_labels = []

for i, batch in enumerate(val_dataloader):
    inputs = batch.to(device)
    model_inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }
    with torch.no_grad():
        scores = model(**model_inputs)
        batch_indices = torch.arange(scores.shape[0])
        score = scores[batch_indices, inputs['score_index']]
        target = inputs['score']
    predictions = (score > 0.5).int()

    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(target.cpu().numpy())

confusion_matrix(all_labels, all_predictions)

array([[384,  40],
       [ 41, 402]])